# Purifying selection - is this reduced on 995F haplotypes? 
## calculate PiN/PiS for 995F, 995S and wt haplotypes
- If we do this per haplotype and take mean, we can get CIs

In [1]:
%run setup.ipynb
import hapclust

In [2]:
# load data
callset_haps = np.load('../data/haps_phase2.npz')
haps = allel.HaplotypeArray(callset_haps['haplotypes'])
pos = allel.SortedIndex(callset_haps['POS'])
ann = callset_haps['ANN']

In [3]:
# chop into gene
start = 2358158
stop = 2431617
loc_vgsc = pos.locate_range(start, stop)
h_vgsc = haps[loc_vgsc]
pos_vgsc = pos[loc_vgsc]
ann_vgsc = ann['Annotation'][loc_vgsc]
h_vgsc

<HaplotypeArray shape=(2092, 2284) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [4]:
#chop into F, S and wt haplotypes
pos_995S = 2422651
pos_995F = 2422652
loc_995S = h_vgsc[pos_vgsc.locate_key(pos_995S)] == 1
loc_995F = h_vgsc[pos_vgsc.locate_key(pos_995F)] == 1
loc_res = loc_995F + loc_995S
loc_wt = ~loc_res

In [5]:
h_vgsc_995F = h_vgsc.compress(loc_995F, axis=1)
h_vgsc_995S = h_vgsc.compress(loc_995S, axis=1)
h_vgsc_wt = h_vgsc.compress(loc_wt, axis=1)

In [6]:
h_vgsc_995F

<HaplotypeArray shape=(2092, 1131) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [7]:
#count alleles
acS = h_vgsc_995S.count_alleles()
acF = h_vgsc_995F.count_alleles()
acw = h_vgsc_wt.count_alleles()
#quick look at first ALT allele
sum(acS[:,1]), sum(acF[:,1]), sum(acw[:,1])

(80372, 183593, 107033)

In [8]:
#get N and S positions
miss_bool = np.asarray([v  == b'missense_variant' for v in ann_vgsc])
pos_vgsc_Non = pos_vgsc.compress(miss_bool)
pos_vgsc_Syn = pos_vgsc.compress(~miss_bool)

In [12]:
acF_Non = acF.compress(miss_bool, axis=0)
acF_Syn = acF.compress(~miss_bool, axis=0)
piFN = np.sum(allel.mean_pairwise_difference(acF_Non, fill=0))
piFS = np.sum(allel.mean_pairwise_difference(acF_Syn, fill=0))
F = piFN/piFS
print(f'995F PiN/PiS = {F}')

995F PiN/PiS = 0.06359678721098909


In [13]:
acS_Non = acS.compress(miss_bool, axis=0)
acS_Syn = acS.compress(~miss_bool, axis=0)
piSN = np.sum(allel.mean_pairwise_difference(acS_Non, fill=0))
piSS = np.sum(allel.mean_pairwise_difference(acS_Syn, fill=0))
S = piSN/piSS
print(f'995S PiN/PiS = {S}')

995S PiN/PiS = 0.0015833883579226286


In [14]:
acw_Non = acw.compress(miss_bool, axis=0)
acw_Syn = acw.compress(~miss_bool, axis=0)
piwN = np.sum(allel.mean_pairwise_difference(acw_Non, fill=0))
piwS = np.sum(allel.mean_pairwise_difference(acw_Syn, fill=0))
wt = piwN/piwS
print(f'wt PiN/PiS = {wt}')

wt PiN/PiS = 0.003173758959776992


In [17]:
F/wt

20.03831671434109

In [19]:
S/wt

0.49890000406139456